In [ ]:
# default_exp models.bases.sequential

# Sequential Base Model
> Implementation of sequential base models in Pytorch Lightning.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.nb_imports import *
from fastcore.test import *

In [ ]:
#export
from typing import Any, Iterable, List, Optional, Tuple, Union, Callable

import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.nn import functional as F

from pytorch_lightning import LightningModule

In [ ]:
#export
class SequentialModel(LightningModule):
    def __init__(self,
                 optimizer='adamw',
                 learning_rate = 0.003,
                 cap = 0,
                 mask = 1,
                 dropout = 0.4,
                 vocab_size = 1000,
                 channels = 128,
                 **kwargs):
        super().__init__()
        self.learning_rate = learning_rate
        self.optimizer = optimizer
        self.cap = cap
        self.mask = mask
        self.dropout = dropout
        self.vocab_size = vocab_size
        self.channels = channels

    def forward(self, users, items):
        raise NotImplementedError

    @staticmethod
    def masked_accuracy(y_pred: torch.Tensor, y_true: torch.Tensor, mask: torch.Tensor):
        _, predicted = torch.max(y_pred, 1)
        y_true = torch.masked_select(y_true, mask)
        predicted = torch.masked_select(predicted, mask)
        acc = (y_true == predicted).double().mean()
        return acc

    @staticmethod
    def masked_ce(y_pred, y_true, mask):
        loss = F.cross_entropy(y_pred, y_true, reduction="none")
        loss = loss * mask
        return loss.sum() / (mask.sum() + 1e-8)

    def training_step(self, batch, batch_idx):
        src_items, y_true = batch
        y_pred = self(src_items)
        y_pred = y_pred.view(-1, y_pred.size(2))
        y_true = y_true.view(-1)
        src_items = src_items.view(-1)
        mask = src_items == self.mask
        loss = self.masked_ce(y_pred=y_pred, y_true=y_true, mask=mask)
        accuracy = self.masked_accuracy(y_pred=y_pred, y_true=y_true, mask=mask)
        self.log("train_loss", loss)
        self.log("train_accuracy", accuracy)
        return loss

    def validation_step(self, batch, batch_idx):
        src_items, y_true = batch
        y_pred = self(src_items)
        y_pred = y_pred.view(-1, y_pred.size(2))
        y_true = y_true.view(-1)
        src_items = src_items.view(-1)
        mask = src_items == self.mask
        loss = self.masked_ce(y_pred=y_pred, y_true=y_true, mask=mask)
        accuracy = self.masked_accuracy(y_pred=y_pred, y_true=y_true, mask=mask)
        self.log("valid_loss", loss)
        self.log("valid_accuracy", accuracy)
        return loss

    def test_step(self, batch, batch_idx):
        src_items, y_true = batch
        y_pred = self(src_items)
        y_pred = y_pred.view(-1, y_pred.size(2))
        y_true = y_true.view(-1)
        src_items = src_items.view(-1)
        mask = src_items == self.mask
        loss = self.masked_ce(y_pred=y_pred, y_true=y_true, mask=mask)
        accuracy = self.masked_accuracy(y_pred=y_pred, y_true=y_true, mask=mask)
        self.log("test_loss", loss)
        self.log("test_accuracy", accuracy)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, patience=10, factor=0.1
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
            "monitor": "valid_loss",
        }